# What do I mean by DSL Algebra?

When I say algebra, I don't think about implementing i.e. DSL for linear algebra in Python. I think about algebra in general as in "a set of objects and a collection of operations on them".

I like algebras, when they are used in programming. Probably the most well-known is Relational Algebra. The object in Relational Algebra is a *relation*, with operations like *join*, *projection* and *union*. The nice thing about having something that works like an algebra is, that you always work with the same type of object, reusing the box of tools without breaking the flow.

That is one of the reasons I like Pythons collections. Pythons sets and list actually have quite a nice algebra built around them.


In [1]:
{1,2,3} | {3,4,5}

{1, 2, 3, 4, 5}

In [2]:
{1,2,3} & {3,4,}

{3}

In [3]:
{1,2,3} - {3,4,5}

{1, 2}

And what I really like is, that all the laws I remember about sets from discrete math still work. For example, with union as `|` and empty set as `set()`:

In [4]:
a = {1,2,3}
a == a|set()==set()|a


True

In [5]:
b = {3,4,5}
c = {5,6,7}
(a | b) | c == a | (b | c)

True

My personal preference is to use these math-like operators over calling methods on objects, even though methpd calls would produce the same result, and often would be more efficient

In [6]:
a = {1,2}
a.add(3)
a.add(4)
a.remove(2)
a == ({1,2} | {3} | {4}) - {2}

True

To be honest, I probably would write this code with method calls, because they are more readable than operators: 

In [7]:
a == {1,2,3}.union({3,4}).difference({2})

True

I don't really want to have operators everywhere, what I want is composability. Now-days we might call this *fluent* interface, but I want to go one step further, I want the interface to conform to predefined set of rules.

This is what I mean when I say algebra:
* interfaces I work with *compose* with each other, they interoperate well, they are *fluent*
* the implementation is *lawful*, it conforms to predefined rules

By **dsl algebra** I mean two distinct, but related things:
* making a dsl that forms an algebra
* creating algebra that helps with creating dsl-s

# Why?

So I have been watching too many talks from Scala conferences and reading too many posts from haskell enthusiasts.
And now I envy them. Because I am mostly working with javascript and python and barely anybody even knows what `monad` *is*, while on the scala-side they are using co-free co-monads to implement cool, extensible DSLs.

Ok, there is fantasy-land and Ramda, but last time my colleague was trying to show off Ramda to the rest of the team, he mostly got puzzled looks from the most of them. And there isn't much in Python.

But I still want to try all of those cool things I see in the typed functional languages.

# Monoids are cool

One useful concept to use here is the one of a *monoid*. 
If we think of this as an interface that is implemented by a type *T*, it needs two things:
* the *empty* element, usually called *mempty*
* the *append* operation, that takes any two T's and returns a new T

There are two laws:
* for any a of type T: append(a,empty) == append(empty,a) == a
* for any a,b,c of type T: append(a,append(b,c)) == append(append(a,b),c)

Now we have this interface, we can do cool things with this :)

And by cool thigs, I mean going through [this presentation](https://www.youtube.com/watch?v=WsA7GtUQeB8) by Gabriel Gonzalez and translating the concepts from haskell to python :)

If you are not scared of Haskell, I really recomend that presentation. There is even a [souce for it](https://github.com/Gabriel439/slides/tree/master/lambdaconf/category).

Typeclasses present the first translation hiccup. In haskell you use these to signidy that a type conforms to some interface, similarily to Java/C# interfaces. Nice thing about this is, that you can define implementations for existing types. This makes them more similiar to C# extension methods, or Clojure protocols. Fortunately, we have single dispatch in Python to simulate this.

In [8]:
from functools import singledispatch
@singledispatch
def mempty(a):
    raise Error("Not implemented for" + a)

@singledispatch
def mappend(a, b):
    raise Error("Not implemented for" + a)

We can definitely implement these for lists.
* mappend is +, and we know that appending lists is associative
* mempty is [], and we know that it doesn't matter if you'd append empty list to left or right

In [9]:
@mempty.register(list)
def _(a):
    return []

@mappend.register(list)
def _(a,b):
    return a + b

In [10]:
mappend([1,2,3],[4,5,6])

[1, 2, 3, 4, 5, 6]

We can trivially implement these for None. It might look kind'a silly, but it will be useful, once we get to combine function.

In [11]:
@mempty.register(None.__class__)
def _(a):
    return None

@mappend.register(None.__class__)
def _(a,b):
    return None

In [12]:
mappend(None,None) == None

True

# Generic functions

Now we can create a generic function that works on all monoids, such as mconcat, that takes a list and appends its contents.

In [13]:
def mconcat(l):
    acc = l[0]
    for x in l[1:]:
        acc = mappend(acc,x)
    return acc

mconcat([[1,2],[3,4,5]])

[1, 2, 3, 4, 5]

As you can see, we have run into our first problem. In theory we should be able to do mconcat of an [] and get the mempty for the lists member type. But python doesn't have typed lists. Well, we see how far will this get us :)

# Nesting

Second thing we could do, is to try to nest these inside of other structures.
For example, if you have n-tuple with monoids, you can prove, that the n-tuple is monoid. I am lazy to write out the proof, so I just defer to the presentation I am copying from :)

In [14]:

@mappend.register((0,0).__class__)
def _(a,b):
    return tuple(mappend(i,j) for i,j in zip(a,b))

In [15]:
mappend(([1,2,3],[10,11,12]),([4,5,6],[14,15]))

([1, 2, 3, 4, 5, 6], [10, 11, 12, 14, 15])

This of course means that we can nest the touples in other touples :)

In [16]:
mappend(([1,2],([10,11],([20,21],[25,26]))),
       ([3,4],([12,13],([22,23],[26,28]))))

([1, 2, 3, 4], ([10, 11, 12, 13], ([20, 21, 22, 23], [25, 26, 26, 28])))

# Nesting with functions

If we have function f that accepts type A as input param and returns type B, then f forms a semigroup if B forms a semigroup.

Basically, we pass the input into all of the functions and then we append the results.

In [34]:
@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    a_r= a(*x)
    b_r=b(*x)
    return mappend(a_r,b_r)
  return result

This means we can send a single arg to multiple functions that return None. This is where the mappend definition comes useful, because without it we would have seen exceptions here.

In [18]:
def phello(arg):
  print("Hello",arg)
  
def phi(arg):
  print("Hi",arg)

mappend(phello,phi)("World!")

Hello World!
Hi World!


Or they could return function that returns None :)

In [19]:
def promptName():
    name = input("Enter Your Name: ")
    return lambda: print("Hi ", name)

def promptAge():
    age = input("Enter Your Age: ")
    return lambda: print("Your Age is", age)

promptBoth = mappend(promptName,promptAge)

In [20]:
answers = promptBoth()

Enter Your Name: Adam
Enter Your Age: 10


In [21]:
answers()

Hi  Adam
Your Age is 10


We could actually get these back, if we return a monoid from tose inner functions.

In [22]:
def getName():
    name = input("Enter Your Name: ")
    return lambda: [name]

def getAge():
    age = input("Enter Your Age: ")
    return lambda: [age]

getBoth = mappend(getName,getAge)
listAnswers = getBoth()
listAnswers()

Enter Your Name: Adam
Enter Your Age: 10


['Adam', '10']

# Function nesting and laws

Lets look again at the most important law that this `mappend` needs to uphold.

```
mappend(f, mappend(g,h)) == mappend(mappend(f,g),h)
```
, given any functions `a`,`b`,`c`, that 
* accept the same input param 
* and return the same semigroup.

To make this more readable, I will use $\oplus$ instead of `mappend`. This means, that I want to prove
$$ f \oplus (g \oplus h) = (f \oplus g) \oplus h $$

Functions are equivalent when for any input, they produce equal output.

So, lets assume arbitrary input of `x`, and we will try to evaluate this on the both sides and try get to the same result (we will mark result of $f(x)$ as $f_x$ and we will evaluate the functions from the left):

$$ \begin{align*} (f \oplus (g \oplus h))(x) &= f(x) \oplus (g \oplus h)(x) \\
&= f_x \oplus (g \oplus h) \\
&= f_x \oplus (g(x) \oplus h(x)) \\
&= f_x \oplus (g_x \oplus h(x) )\\
&= f_x \oplus (g_x \oplus h_x)
\end{align*} $$

$$ \begin{align*} ((f \oplus g) \oplus h)(x) &= (f \oplus g)(x) \oplus h(x) \\
&= (f(x) \oplus g(x)) \oplus h(x) \\
&= (f_x \oplus g(x)) \oplus h(x) \\
&= (f_x \oplus g_x) \oplus h(x) \\
&= (f_x \oplus g_x) \oplus h_x 
\end{align*} $$

Because we know, that $f_x$, $g_x$ and $h_x$ are from the same semigroup, the last lines are equivalent. There is one big assumption, and that is, that the function application will always start at right. This way, no matter the bracket position, $f(x)$ in our example always gets evaluated first.  This is why we can't i.e. flip the evaluation of the functions in our implementation, because ordering matters, and next code sample would introduce a subtle bug.

```
@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    b_r = b(*x)
    a_r = a(*x)
    return mappend(a_r,b_r)
  return result
```

In math, we know that function will always return the same result for the same input, in python this is not the case. This means, that while sketching out a proof is helpful, we can't rely on it too much and still need to be doing testing.

# Dsl for getting configuration

We could make a little library for asking for config from this :)

If you have read Gabriels presentation, you might think, that now I would be explaining Monads, and then creating monoidal event stream combinator. Unfortunately, I don't think I am hard-core enough to do that in python.

On the other hand, writing a tiny config library sounds like a bit of harmless fun.

First, lets create a mappend for Dictionary.

In [23]:
@mappend.register(dict)
def _(a,b):
    return {**a, **b}

Now we could create a generic function, i.e. askFor:

In [24]:
def askFor(name):
    def getAnswer():
        answer = input(name)
        return {name: answer}
    return getAnswer

In [25]:
askAll = mconcat([
        askFor('name'),
        askFor('age'),
        askFor('email')
    ])

In [26]:
askAll()

nameAdam
age10
emaila@a.a


{'age': '10', 'email': 'a@a.a', 'name': 'Adam'}

What we could do now, instead of asking for input manually, we could pass in a config string and parse it. I will first create the config string, with a simple structure "key:value" on each line.

In [27]:
config = """
name:eve
age:16
dance:swing
"""

Now I can create a simple parsing function, where I can input the key, and it will return the value.
Actually, it will return a parser that takes the string produces a dictionary with single KV pair, but thats almost the same, just more composable :)

And for now, if it doesn't find the key, it produces empty dictionary.

In [28]:
import re

def parseFor(name):
    def getAnswer(config):
        m = re.search('(?<='+name+').*', config)
        if m == None:
            return {}
        else: 
            return {name: m.group(0)}
    return getAnswer

In [29]:
parseAll = mconcat([
        parseFor('name'),
        parseFor('age'),
        parseFor('email')
    ])

parseAll(config)

{'age': ':16', 'name': ':eve'}

And because in our parsing functions we get the whole config, we could parse different things as well. For example we could get names of all of the keys.

In [30]:
def getKeys(config):
    return {"keys": [x.strip() for x in re.split(":.*\n",config) if x.strip()!= ""]}

In [33]:
parseAll = mconcat([parseFor('name'),getKeys])

parseAll(config)

{'keys': ['name', 'age', 'dance'], 'name': ':eve'}

## Better config object?

Another observation we can make, is that we can create an alternative interpretation of dicitonary mappend, where don't just over-write the latter, but we assume that values of the dictionary are monoids themselves. This means we can mappend them again :-)

In [36]:
@mappend.register(dict)
def _(a,b):
    return {**a, **b,**{k:mappend(a[k],b[k]) for k in a if k in b}}
    

We could then have additional alternative implementation for our function mappend.
Our new mappend would return the result of the first function, that doesn't return `none`.

In [37]:
@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    a_r= a(*x)
    if a_r!=None:
        return a_r;
    b_r=b(*x)
    return b_r
  return result

This way we can flip the building blocks for our little dsl.

In [44]:
def askFor(name):
    def getAnswer(config):
        answer = input(name)
        return answer
    return {name: getAnswer}

def parseFor(name):
    def getAnswer(config):
        m = re.search('(?<='+name+').*', config)
        if m == None:
            return None 
        return m.group(0)
    return {name: getAnswer}



In [45]:
parseAll = mconcat([
        parseFor('name'),
        askFor('name'),
        parseFor('age'),
        askFor('age'),
        parseFor('email'),
        askFor('email')
    ])

parseAll

{'age': <function __main__._.<locals>.result>,
 'email': <function __main__._.<locals>.result>,
 'name': <function __main__._.<locals>.result>}

The drawback is, that now we have dictionary of functions, where previously we had just a singe funciton that got us our config. We an still implement that:

In [46]:
def getConfig(parser, config):
    return {k:parser[k](config) for k in parser}

emaila@a.a


{'age': ':16', 'email': 'a@a.a', 'name': ':eve'}

Way we implemented this (dictionary keys get mappended and the first return from a funcion wins) we should only be prompted for the email, and rest of this should be parsed from config.

In [47]:
getConfig(parseAll,config)

emaila@a.c


{'age': ':16', 'email': 'a@a.c', 'name': ':eve'}

And now I almost have what I wanted in the beginning. I have 
* created a dsl (for parsing configuration)
* the building blocks are nice, and I can just `mconcat` them together
* if I want to add new building block, it just needs to
  * accept the config-string
  * return a KV dict
  
 On the other hand, tit has no error handling, it is kind-of inefficient, and in practce would be just a useless toy. I think the solution here is more category theory, obviously! 
 
One thing I didn't really talk about are applicatives, that havw been heavily referenced in Gabriels talk. Thing is, in haskel, many libraries have applicative interface. In python, I would first need to figure out, what would that even mean for a type.